<a href="https://www.kaggle.com/victororlov/synopsis-keyword-extraction?scriptVersionId=89521420" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Install and import modules

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re

from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
from tqdm.notebook import tqdm
import ast
from matplotlib import pyplot as plt

In [2]:
%%capture
!pip install keybert

In [3]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [4]:
from keybert import KeyBERT
kw_model = KeyBERT()

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Load and preprosess data

In [5]:
df = pd.read_csv('../input/myanimelist-dataset-animes-profiles-reviews/animes.csv')
df = df[['synopsis', 'title']]
df.dropna(inplace=True)
df = df[~df.title.duplicated(keep='first')]
df.head()

,synopsis,title
0,Following their participation at the Inter-Hig...,Haikyuu!! Second Season
1,Music accompanies the path of the human metron...,Shigatsu wa Kimi no Uso
2,The Abyss—a gaping chasm stretching down into ...,Made in Abyss
3,"""In order for something to be obtained, someth...",Fullmetal Alchemist: Brotherhood
4,After helping revive the legendary vampire Kis...,Kizumonogatari III: Reiketsu-hen


Let's take a popular anime series 'Shingeki no Kyojin' for example:

In [6]:
TITLE = 'Shingeki no Kyojin'
text = df[df['title'] == TITLE].synopsis.values[0]
def clean_up_text(text):
    doc = re.sub("[\(\[].*?[\)\]]", "", text) # Remove the "written by" caption
    doc = doc.replace(u'\n', u'').replace(u'\r', u'')
    doc = nlp(doc)
    return doc

doc = clean_up_text(text)
print(doc)

Centuries ago, mankind was slaughtered to near extinction by monstrous humanoid creatures called titans, forcing humans to hide in fear behind enormous concentric walls. What makes these giants truly terrifying is that their taste for human flesh is not born out of hunger but what appears to be out of pleasure. To ensure their survival, the remnants of humanity began living within defensive barriers, resulting in one hundred years without a single titan encounter. However, that fragile calm is soon shattered when a colossal titan manages to breach the supposedly impregnable outer wall, reigniting the fight for survival against the man-eating abominations.  After witnessing a horrific personal loss at the hands of the invading creatures, Eren Yeager dedicates his life to their eradication by enlisting into the Survey Corps, an elite military unit that combats the merciless humanoids outside the protection of the walls. Based on Hajime Isayama's award-winning manga,  Shingeki no Kyojin  

# Extract keyword candidates

OK, now we need to create a list of all possible words and phrases in the text that could be a keyword. Let's call them candidates. In [[1]](#ref) Maarten Grootendorst proposes using n-grams. But I find the resulting candidates too nonsensical as this approach does take grammatical structure into account and treats all consecutive strings of words of certan lenght as a candidate.

So I have tried something different: the spaCy module knows how to decompose the sentence into grammatical pieces 

In [7]:
for npf in doc.noun_chunks: # use np instead of np.text
    if len(npf) > 1:
        print('> ', npf)

>  monstrous humanoid creatures
>  enormous concentric walls
>  these giants
>  their taste
>  human flesh
>  their survival
>  the remnants
>  defensive barriers
>  one hundred years
>  a single titan encounter
>  that fragile calm
>  a colossal titan
>  the supposedly impregnable outer wall
>  the fight
>  the man-eating abominations
>  a horrific personal loss
>  the hands
>  the invading creatures
>  Eren Yeager
>  his life
>  their eradication
>  the Survey Corps
>  an elite military unit
>  the merciless humanoids
>  the protection
>  the walls
>  Hajime Isayama's award-winning manga
>  his adopted sister
>  Mikasa Ackerman
>  his childhood friend
>  Armin Arlert
>  the brutal war
>  the titans
>  a way
>  the last walls


Ok, that sound too truncated. Let's try something different

In [8]:
# Based on https://stackoverflow.com/questions/48925328/how-to-get-all-noun-phrases-in-spacy
def get_keyword_candidates(doc):
    # code to recursively combine nouns
    # 'We' is actually a pronoun but included in your question
    # hence the token.pos_ == "PRON" part in the last if statement
    # suggest you extract PRON separately like the noun-chunks above

    index = 0
    nounIndices = []
    for token in doc:
        if token.pos_ == 'NOUN':
            nounIndices.append(index)
        index = index + 1

    print('Nouns found: ', len(nounIndices))

    candidates = []
    for idxValue in nounIndices:
        if not bool(doc[idxValue].left_edge.ent_type_):
            start = doc[idxValue].left_edge.i
        else:
            start = idxValue 

        if not bool(doc[idxValue].right_edge.ent_type_):
            finish = doc[idxValue].right_edge.i+1
        else:
            finish = idxValue + 1

        if finish-start > 0 and finish-start <7:
            span = doc[start : finish]
#             print('>', span)
            candidates.append(span.text)

    return candidates

candidates = get_keyword_candidates(doc)
print(candidates)

Nouns found:  45
['Centuries', 'mankind', 'humanoid', 'monstrous humanoid creatures called titans', 'titans', 'humans', 'fear', 'enormous concentric walls', 'these giants', 'their taste for human flesh', 'human flesh', 'hunger', 'pleasure', 'their survival', 'the remnants of humanity', 'humanity', 'defensive barriers', 'years', 'a single titan encounter', 'that fragile calm', 'the supposedly impregnable outer wall', 'man', 'the man-eating abominations', 'a horrific personal loss', 'the hands of the invading creatures', 'the invading creatures', 'his life', 'their eradication', 'merciless', 'the protection of the walls', 'the walls', 'award', 'manga,  ', 'his adopted sister', 'his childhood friend', 'the titans and race', 'race', 'the last walls']


In [9]:
keywords = kw_model.extract_keywords(doc.text, candidates=candidates, 
                              use_mmr=True, diversity=0.7)

print(keywords)

[('monstrous humanoid creatures called titans', 0.5308), ('the protection of the walls', 0.3215), ('hunger', 0.2733), ('their eradication', 0.3178), ('merciless', 0.166)]


<a id='ref'></a>
# References
1. Maarten Grootendorst (2020). [Keyword Extraction with BERT](https://towardsdatascience.com/keyword-extraction-with-bert-724efca412ea)